# Quickstart: Build, test, and deploy an agent using Mosaic AI Agent Framework
This quickstart notebook demonstrates how to build, test, and deploy a generative AI agent ([AWS](https://docs.databricks.com/aws/en/generative-ai/guide/introduction-generative-ai-apps#what-are-gen-ai-apps) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/guide/introduction-generative-ai-apps#what-are-gen-ai-apps) | [GCP](https://docs.databricks.com/gcp/en/generative-ai/guide/introduction-generative-ai-apps)) using Mosaic AI Agent Framework ([AWS](https://docs.databricks.com/aws/en/generative-ai/agent-framework/build-genai-apps#-mosaic-ai-agent-framework) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/agent-framework/build-genai-apps#-mosaic-ai-agent-framework) | [GCP](https://docs.databricks.com/gcp/en/generative-ai/agent-framework/build-genai-apps#-mosaic-ai-agent-framework)) on Databricks

# Building Your First AI Agent with Databricks Mosaic AI: A Complete Guide

## Introduction

Welcome to this comprehensive walkthrough of building, testing, and deploying a production-ready AI agent using Databricks' Mosaic AI Agent Framework! Whether you're a data engineer, ML practitioner, or solution architect working with Databricks, this guide will take you through every step of creating an intelligent agent that can interact with users and execute code to solve problems.

In this tutorial, we'll build an agent that:
- Uses Large Language Models (LLMs) served on Databricks Foundation Model API
- Can execute Python code dynamically to solve computational problems
- Is fully deployable as a production endpoint
- Integrates seamlessly with MLflow for tracking and versioning

Let's dive into each cell of this quickstart notebook and understand exactly what's happening under the hood!

---

## Part 1: Setting Up the Environment

### Cell 1: Installing Required Dependencies

```python
%pip install -U -qqqq mlflow databricks-openai databricks-agents
dbutils.library.restartPython()
```

**What's happening here?**

This cell installs three critical Python packages:

1. **`mlflow`**: The open-source platform for managing the ML lifecycle, including experimentation, reproducibility, and deployment. We'll use this to log, version, and deploy our agent.

2. **`databricks-openai`**: A specialized SDK that provides an OpenAI-compatible interface for interacting with Databricks model serving endpoints. This allows us to use familiar OpenAI-style APIs while leveraging Databricks' infrastructure.

3. **`databricks-agents`**: The core framework for building and deploying agents on Databricks, providing tools for agent orchestration and deployment.

The `-U` flag ensures we get the latest versions, and `-qqqq` suppresses verbose installation output for a cleaner notebook experience.

**Why restart Python?**

The `dbutils.library.restartPython()` command is crucial—it restarts the Python interpreter to ensure all newly installed packages are properly loaded into the notebook's runtime environment. This prevents import errors and version conflicts.

---

## Part 2: Selecting the Right LLM Endpoint

### Cell 2: Dynamic LLM Endpoint Selection

```python
LLM_ENDPOINT_NAME = None

from databricks.sdk import WorkspaceClient

def is_endpoint_available(endpoint_name):
    try:
        client = WorkspaceClient().serving_endpoints.get_open_ai_client()
        client.chat.completions.create(
            model=endpoint_name, 
            messages=[{"role": "user", "content": "What is AI?"}]
        )
        return True
    except Exception:
        return False

client = WorkspaceClient()
for candidate_endpoint_name in ["databricks-claude-3-7-sonnet", "databricks-meta-llama-3-3-70b-instruct"]:
    if is_endpoint_available(candidate_endpoint_name):
        LLM_ENDPOINT_NAME = candidate_endpoint_name
        
assert LLM_ENDPOINT_NAME is not None, "Please specify LLM_ENDPOINT_NAME"
```

**Understanding the `is_endpoint_available()` function:**

This elegant function performs a "health check" on an LLM endpoint by:

1. **Creating a WorkspaceClient**: This establishes a connection to your Databricks workspace with automatic authentication
2. **Getting an OpenAI client**: Retrieves a client configured to communicate with Databricks serving endpoints
3. **Sending a test query**: Makes a simple chat completion request with "What is AI?" to verify the endpoint responds
4. **Returning True/False**: If the request succeeds, the endpoint is available; any exception means it's not

**The smart fallback mechanism:**

The code iterates through a list of candidate LLM endpoints:
- **Claude 3.7 Sonnet**: Anthropic's powerful model known for following instructions precisely
- **Meta Llama 3.3 70B Instruct**: Meta's open-source instruction-tuned model

This approach ensures your notebook works across different Databricks workspace configurations. If your workspace doesn't have Claude available, it automatically falls back to Llama 3.3.

**The assertion at the end** guarantees that we have a working endpoint before proceeding—failing fast if no LLM is available.

---

## Part 3: Building the Agent Core Logic

### Cell 3: Agent Implementation with Tool Calling

This is the heart of our agent! Let's break it down function by function:

```python
import json
import mlflow
from databricks.sdk import WorkspaceClient
from databricks_openai import UCFunctionToolkit, DatabricksFunctionClient

# Automatically log traces from LLM calls for ease of debugging
mlflow.openai.autolog()
```

**MLflow Autologging:**

`mlflow.openai.autolog()` is incredibly powerful—it automatically captures:
- Every LLM request and response
- Token usage and costs
- Latency metrics
- Tool calls made during execution

This gives you complete observability into your agent's behavior without writing manual logging code.

---

#### Function 1: Setting Up the OpenAI Client

```python
# Get an OpenAI client configured to talk to Databricks model serving endpoints
openai_client = WorkspaceClient().serving_endpoints.get_open_ai_client()
```

This creates a client that speaks the OpenAI API protocol but routes requests to Databricks' infrastructure—giving you the best of both worlds: familiar APIs and enterprise-grade serving.

---

#### Function 2: Loading Built-in Tools

```python
# Load Databricks built-in tools (a stateless Python code interpreter tool)
client = DatabricksFunctionClient()
builtin_tools = UCFunctionToolkit(
    function_names=["system.ai.python_exec"], 
    client=client
).tools

for tool in builtin_tools:
    del tool["function"]["strict"]
```

**What's happening here?**

- **`DatabricksFunctionClient`**: Creates a client for accessing Unity Catalog functions
- **`UCFunctionToolkit`**: Loads tools registered in Unity Catalog—in this case, `system.ai.python_exec`, a secure Python code execution environment
- **Removing "strict" parameter**: This line removes strict schema validation to ensure compatibility with different LLM providers

**Why is this powerful?**

The `system.ai.python_exec` tool allows your agent to write and execute Python code dynamically to solve problems—like a data scientist working through computations in a notebook!

---

#### Function 3: `call_tool()` - Executing Agent Tools

```python
def call_tool(tool_name, parameters):
    if tool_name == "system__ai__python_exec":
        return DatabricksFunctionClient().execute_function(
            "system.ai.python_exec", 
            parameters=parameters
        )
    raise ValueError(f"Unknown tool: {tool_name}")
```

**Purpose:**

This function acts as a **router** that maps tool names to their execution logic.

**Key details:**

- **Tool name normalization**: Note the double underscores (`system__ai__python_exec`) replacing dots—this follows OpenAI's function calling naming conventions
- **Parameter passing**: The function receives parameters as a dictionary and forwards them to the Unity Catalog function executor
- **Error handling**: Raises a clear error if an unknown tool is requested
- **Extensibility**: You can easily add more tools by adding additional `if` conditions

---

#### Function 4: `run_agent()` - The Orchestration Engine

```python
def run_agent(prompt):
    """
    Send a user prompt to the LLM, and return a list of LLM response messages
    The LLM is allowed to call the code interpreter tool if needed, to respond to the user
    """
    result_msgs = []
    
    # Step 1: Send the prompt to the LLM with available tools
    response = openai_client.chat.completions.create(
        model=LLM_ENDPOINT_NAME,
        messages=[{"role": "user", "content": prompt}],
        tools=builtin_tools,
    )
    
    msg = response.choices[0].message
    result_msgs.append(msg.to_dict())

    # Step 2: If the model executed a tool, call it
    if msg.tool_calls:
        call = msg.tool_calls[0]
        tool_result = call_tool(
            call.function.name, 
            json.loads(call.function.arguments)
        )
        result_msgs.append({
            "role": "tool",
            "content": tool_result.value,
            "name": call.function.name,
            "tool_call_id": call.id,
        })
        
    return result_msgs
```

**Deep dive into this critical function:**

**Step 1: LLM Invocation**
- Creates a chat completion request with the user's prompt
- Crucially includes `tools=builtin_tools`, informing the LLM it can call the Python executor
- The LLM decides whether it needs to use a tool to answer the question

**Step 2: Tool Execution (Conditional)**
- Checks if `msg.tool_calls` exists—this means the LLM decided it needs to execute code
- Extracts the tool name and arguments from the first tool call
- Calls our `call_tool()` function to execute the code
- Appends the tool's result to the message history with proper formatting

**Return value:**
A list of messages representing the conversation flow—this could be just the LLM's direct answer, or an LLM response + tool call + tool result sequence.

**Important limitation:**
This implementation handles only **one tool call** per prompt (note `tool_calls[0]`). For production agents, you'd want to loop through multiple tool calls and potentially allow multi-turn conversations.

---

#### Testing the Agent

```python
answer = run_agent("What is the square root of 429?")
for message in answer:
    print(f'{message["role"]}: {message["content"]}')
```

**What happens when we run this:**

1. The LLM receives: "What is the square root of 429?"
2. It recognizes this requires computation and generates a tool call to `python_exec`
3. The tool executes Python code like: `import math; math.sqrt(429)`
4. The result (~20.71) is returned
5. We print both the assistant's tool call and the tool's result

**Expected output:**
```
assistant: [Tool call to execute Python code]
tool: 20.71231... (the actual result)
```

---

## Part 4: MLflow Integration - Making it Production-Ready

### Cell 4: Wrapping in MLflow's ChatAgent Interface

```python
import uuid
import mlflow
from typing import Any, Optional
from mlflow.pyfunc import ChatAgent
from mlflow.types.agent import ChatAgentMessage, ChatAgentResponse, ChatContext

mlflow.openai.autolog()

class QuickstartAgent(ChatAgent):
    def predict(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> ChatAgentResponse:
        # 1. Extract the last user prompt from the input messages
        prompt = messages[-1].content
        
        # 2. Call run_agent to get back a list of response messages
        raw_msgs = run_agent(prompt)
        
        # 3. Map each response message into a ChatAgentMessage and return
        out = []
        for m in raw_msgs:
            out.append(ChatAgentMessage(id=uuid.uuid4().hex, **m))
            
        return ChatAgentResponse(messages=out)
```

**Why wrap in MLflow's ChatAgent?**

MLflow's `ChatAgent` is a standardized interface that provides:
- **Consistent API**: All agents follow the same prediction pattern
- **Built-in UI**: Automatic chat interfaces in Databricks AI Playground
- **Deployment support**: Seamless deployment to model serving endpoints
- **Tracking**: Integration with MLflow's experiment tracking

**Understanding the `predict()` method:**

**Parameters:**
- **`messages`**: A list of `ChatAgentMessage` objects representing the conversation history
- **`context`**: Optional additional context (documents, metadata, etc.)
- **`custom_inputs`**: Custom parameters for specialized behavior

**Implementation logic:**

1. **Extract the latest prompt** (`messages[-1].content`): Gets the most recent user message from the conversation
2. **Call our agent logic**: Invokes `run_agent()` with the prompt
3. **Format responses**: Converts raw message dictionaries into `ChatAgentMessage` objects with unique IDs
4. **Return standardized response**: Wraps messages in `ChatAgentResponse` for MLflow compatibility

**Why generate UUIDs?**
Each message needs a unique identifier for tracking, debugging, and UI rendering purposes.

---

#### Testing the MLflow Agent

```python
AGENT = QuickstartAgent()

for response_message in AGENT.predict(
    {"messages": [{"role": "user", "content": "What's the square root of 429?"}]}
).messages:
    print(f"role: {response_message.role}, content: {response_message.content}")
```

This tests that our MLflow-wrapped agent works identically to the raw implementation, but now follows MLflow's standard interface.

---

## Part 5: Logging the Agent for Deployment

### Cell 5: Writing Agent Code to a File

```python
%%writefile quickstart_agent.py
[... entire agent implementation ...]
mlflow.models.set_model(AGENT)
```

**The magic of `%%writefile`:**

This Jupyter/Databricks notebook magic command writes the entire cell content to a file named `quickstart_agent.py`. This is crucial because:

1. **MLflow needs a Python file** to package and deploy
2. **All dependencies must be in one place** for containerization
3. **The code becomes versionable** as a standalone artifact

**Key addition: `mlflow.models.set_model(AGENT)`**

This line tells MLflow: "When someone loads this model, instantiate the `AGENT` object and use it for predictions." It's the bridge between code and deployable model.

**Complete agent structure in the file:**
- All imports
- Endpoint selection logic
- Tool definitions
- `call_tool()` function
- `run_agent()` function
- `QuickstartAgent` class
- Model registration call

---

### Cell 6: Registering the Agent to Unity Catalog

```python
dbutils.library.restartPython()

import mlflow
from mlflow.models.resources import DatabricksFunction, DatabricksServingEndpoint
from pkg_resources import get_distribution
from quickstart_agent import LLM_ENDPOINT_NAME
```

**Why restart Python again?**

After writing `quickstart_agent.py`, we restart to ensure we can import it cleanly as a module.

---

#### Setting up Model Registry Location

```python
# Register the model to the workspace default catalog.
catalog_name = spark.sql("SELECT current_catalog()").collect()[0][0]
schema_name = "default"
registered_model_name = f"{catalog_name}.{schema_name}.quickstart_agent"
```

**Unity Catalog three-level namespace:**

Databricks uses a hierarchical structure:
- **Catalog**: Top-level container (like a database in traditional systems)
- **Schema**: Logical grouping within a catalog (like a schema in PostgreSQL)
- **Model**: The actual registered model artifact

**Dynamic catalog detection:**

`spark.sql("SELECT current_catalog()")` retrieves your workspace's current catalog, making the code portable across environments.

**Example result:** `"main.default.quickstart_agent"`

---

#### Specifying Required Resources

```python
# Specify Databricks product resources that the agent needs access to
resources = [
    DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME),
    DatabricksFunction(function_name="system.ai.python_exec"),
]
```

**Why declare resources?**

This is one of Databricks' most powerful features for production deployment:

1. **Automatic authentication**: Databricks configures service principals and tokens automatically
2. **Dependency tracking**: The platform knows exactly what services your agent needs
3. **Access control**: Ensures the deployed agent has proper permissions
4. **Governance**: Auditing and compliance teams can see all resource dependencies

**Resources in this agent:**
- **LLM Endpoint**: The model serving endpoint for Claude or Llama
- **Python Executor**: The Unity Catalog function for code execution

---

#### Logging and Registering the Model

```python
mlflow.set_registry_uri("databricks-uc")

with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        artifact_path="agent",
        python_model="quickstart_agent.py",
        extra_pip_requirements=[
            f"databricks-connect=={get_distribution('databricks-connect').version}"
        ],
        resources=resources,
        registered_model_name=registered_model_name,
    )
```

**Breaking down each parameter:**

**`mlflow.set_registry_uri("databricks-uc")`**
- Tells MLflow to use Unity Catalog (UC) as the model registry
- Enables enterprise features like access control and lineage

**`mlflow.start_run()`**
- Creates an MLflow experiment run to track this model logging operation
- All metrics, parameters, and artifacts are associated with this run

**`mlflow.pyfunc.log_model()` parameters:**

1. **`artifact_path="agent"`**: Directory name within the MLflow run where artifacts are stored

2. **`python_model="quickstart_agent.py"`**: The Python file containing our agent (MLflow's "models from code" feature)

3. **`extra_pip_requirements`**: Additional Python packages needed at serving time
   - We include `databricks-connect` matching the current version for compatibility

4. **`resources`**: The Databricks resources we defined earlier (endpoints and functions)

5. **`registered_model_name`**: The three-level namespace where the model is registered in Unity Catalog

**Return value:**

`logged_agent_info` contains metadata about the logged model, including:
- Model URI
- Registered model version
- Run ID
- Artifact location

---

## Part 6: Deploying to Production

### Cell 7: Creating a Serving Endpoint

```python
from databricks import agents

deployment_info = agents.deploy(
    model_name=registered_model_name,
    model_version=logged_agent_info.registered_model_version,
    deploy_feedback_model=False,
)
```

**The final step: Deployment!**

**`agents.deploy()` parameters:**

1. **`model_name`**: The Unity Catalog model name we registered (e.g., "main.default.quickstart_agent")

2. **`model_version`**: The specific version number returned from `log_model()` (ensures we deploy exactly what we just logged)

3. **`deploy_feedback_model=False`**: Whether to deploy an additional endpoint for collecting user feedback
   - Set to `False` for this quickstart to simplify deployment
   - In production, you'd want this `True` to gather feedback for evaluation

**What happens during deployment:**

1. **Container creation**: Databricks packages your agent code, dependencies, and environment into a container
2. **Resource provisioning**: Allocates compute resources for serving requests
3. **Authentication setup**: Configures automatic credentials for accessing the LLM endpoint and tools
4. **Health checks**: Verifies the endpoint can serve predictions
5. **Endpoint activation**: Makes the agent available via REST API

**Post-deployment:**

Once deployed, you can:
- **Test in AI Playground**: Interactive chat interface for manual testing
- **Share with stakeholders**: Generate review links for feedback collection
- **Query via API**: Make programmatic requests to the endpoint
- **Monitor performance**: Track latency, throughput, and errors in real-time

---

## Architecture Summary: How It All Fits Together

Here's the complete flow of your deployed agent:

```
User Input
    ↓
[Deployed Agent Endpoint]
    ↓
QuickstartAgent.predict() ← MLflow interface
    ↓
run_agent() ← Orchestration logic
    ↓
OpenAI Client → [LLM Endpoint] (Claude/Llama)
    ↓
LLM decides: Tool needed?
    ├─ No → Return answer directly
    └─ Yes → Generate tool call
        ↓
call_tool() ← Route to correct tool
    ↓
DatabricksFunctionClient → [system.ai.python_exec]
    ↓
Execute Python code securely
    ↓
Return result
    ↓
Format as ChatAgentResponse
    ↓
Return to user
```

---

## Key Takeaways and Best Practices

### 1. Modularity is Key
The code cleanly separates concerns:
- Tool definitions
- Agent orchestration
- MLflow integration
- Deployment configuration

### 2. MLflow Autologging
Always enable `mlflow.openai.autolog()` for production agents—the observability is invaluable for debugging and optimization.

### 3. Resource Declaration
Explicitly declaring Databricks resources ensures:
- Automatic authentication
- Clear dependency tracking
- Proper governance

### 4. Dynamic Endpoint Selection
The fallback mechanism for LLM endpoints makes your code robust across different workspace configurations.

### 5. Standardized Interfaces
Using MLflow's `ChatAgent` interface ensures compatibility with Databricks' agent ecosystem and UIs.

---

## Production Enhancements to Consider

While this quickstart is excellent for learning, here are enhancements for production:

### 1. Multi-turn Conversations
Currently, the agent processes single prompts. Add conversation memory:
```python
def run_agent(messages_history):  # Accept full history
    response = openai_client.chat.completions.create(
        model=LLM_ENDPOINT_NAME,
        messages=messages_history,  # Use full context
        tools=builtin_tools,
    )
```

### 2. Multiple Tool Calls
Handle scenarios where the LLM makes multiple tool calls:
```python
if msg.tool_calls:
    for call in msg.tool_calls:  # Loop through all calls
        tool_result = call_tool(call.function.name, ...)
        # Append each result
```

### 3. Error Handling
Add robust error handling for tool execution:
```python
try:
    tool_result = call_tool(...)
except Exception as e:
    # Return error message to LLM for recovery
    tool_result = f"Error: {str(e)}"
```

### 4. Custom Tools
Extend beyond Python execution:
```python
builtin_tools = UCFunctionToolkit(
    function_names=[
        "system.ai.python_exec",
        "catalog.schema.custom_search_tool",
        "catalog.schema.data_retrieval_tool",
    ],
    client=client
).tools
```

### 5. Prompt Engineering
Add system prompts for better behavior:
```python
messages = [
    {"role": "system", "content": "You are a helpful data analysis assistant..."},
    {"role": "user", "content": prompt}
]
```

### 6. Streaming Responses
For better user experience, implement streaming:
```python
response = openai_client.chat.completions.create(
    model=LLM_ENDPOINT_NAME,
    messages=messages,
    tools=builtin_tools,
    stream=True,  # Enable streaming
)
```

---

## Conclusion

Congratulations! You now understand every aspect of building a production-ready AI agent on Databricks. From installing dependencies to deploying a fully-functional endpoint, you've learned:

✅ How to set up the Databricks Agent Framework  
✅ Dynamic LLM endpoint selection and fallback logic  
✅ Building agents with tool-calling capabilities  
✅ MLflow integration for tracking and deployment  
✅ Unity Catalog registration for governance  
✅ Production deployment with automatic authentication  

This quickstart provides a solid foundation for building more sophisticated agents that can:
- Query databases
- Retrieve documents
- Call custom APIs
- Perform complex data analysis
- Interact with external systems

The Databricks Mosaic AI Agent Framework handles the infrastructure complexity, letting you focus on building intelligent, valuable agents for your organization.

**Next steps:**
- Experiment with different LLM models
- Add custom tools from your Unity Catalog
- Implement evaluation pipelines
- Collect user feedback for continuous improvement
- Scale to handle production traffic

Happy building! 🚀

---

## Resources

- [Databricks Agent Framework Documentation](https://docs.databricks.com/en/generative-ai/agent-framework/)
- [MLflow ChatAgent API](https://mlflow.org/docs/latest/api_reference/python_api/mlflow.pyfunc.html#mlflow.pyfunc.ChatAgent)
- [Unity Catalog Functions](https://docs.databricks.com/en/sql/language-manual/sql-ref-functions-builtin-ai.html)
- [Databricks Foundation Model APIs](https://docs.databricks.com/en/machine-learning/foundation-model-apis/)

---

*This blog post provides a comprehensive breakdown of the Databricks Agent Framework quickstart notebook, explaining each cell, function, and architectural decision in detail. Perfect for data professionals looking to understand and extend AI agent capabilities on Databricks!*

---

**Author Notes:**
- Tutorial source: [Databricks Agent Quickstart](https://docs.databricks.com/aws/en/notebooks/source/generative-ai/agent-quickstart.html)
- Target audience: Data engineers, ML engineers, solution architects
- Difficulty level: Intermediate
- Estimated reading time: 25-30 minutes

## Define and test an agent
This section defines and tests a simple agent with the following attributes:

- The agent uses an LLM served on Databricks Foundation Model API ([AWS](https://docs.databricks.com/aws/en/machine-learning/foundation-model-apis) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/machine-learning/foundation-model-apis/) | [GCP](https://docs.databricks.com/gcp/en/machine-learning/foundation-model-apis))
- The agent has access to a single tool, the built-in Python code interpreter tool on Databricks Unity Catalog. It can use this tool to run LLM-generated code in order to respond to user questions. ([AWS](https://docs.databricks.com/aws/en/generative-ai/agent-framework/code-interpreter-tools#built-in-python-executor-tool) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/agent-framework/code-interpreter-tools) | [GCP](https://docs.databricks.com/gcp/en/generative-ai/agent-framework/code-interpreter-tools))

We will use `databricks_openai` SDK ([AWS](https://docs.databricks.com/aws/en/generative-ai/agent-framework/author-agent#requirements) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/agent-framework/author-agent#requirements) | [GCP](https://docs.databricks.com/gcp/en/generative-ai/agent-framework/author-agent#requirements)) to query the LLM endpoint.

In [0]:
%pip install -U -qqqq mlflow databricks-openai databricks-agents
dbutils.library.restartPython()

In [0]:
# The snippet below tries to pick the first LLM API available in your Databricks workspace
# from a set of candidates. You can override and simplify it
# to just specify LLM_ENDPOINT_NAME.
LLM_ENDPOINT_NAME = None

from databricks.sdk import WorkspaceClient
def is_endpoint_available(endpoint_name):
  try:
    client = WorkspaceClient().serving_endpoints.get_open_ai_client()
    client.chat.completions.create(model=endpoint_name, messages=[{"role": "user", "content": "What is AI?"}])
    return True
  except Exception:
    return False
  
client = WorkspaceClient()
for candidate_endpoint_name in ["databricks-claude-3-7-sonnet", "databricks-meta-llama-3-3-70b-instruct"]:
    if is_endpoint_available(candidate_endpoint_name):
      LLM_ENDPOINT_NAME = candidate_endpoint_name
assert LLM_ENDPOINT_NAME is not None, "Please specify LLM_ENDPOINT_NAME"

In [0]:
import json
import mlflow
from databricks.sdk import WorkspaceClient
from databricks_openai import UCFunctionToolkit, DatabricksFunctionClient

# Automatically log traces from LLM calls for ease of debugging
mlflow.openai.autolog()

# Get an OpenAI client configured to talk to Databricks model serving endpoints
# We'll use this to query an LLM in our agent
openai_client = WorkspaceClient().serving_endpoints.get_open_ai_client()

# Load Databricks built-in tools (a stateless Python code interpreter tool)
client = DatabricksFunctionClient()
builtin_tools = UCFunctionToolkit(
    function_names=["system.ai.python_exec"], client=client
).tools
for tool in builtin_tools:
    del tool["function"]["strict"]


def call_tool(tool_name, parameters):
    if tool_name == "system__ai__python_exec":
        return DatabricksFunctionClient().execute_function(
            "system.ai.python_exec", parameters=parameters
        )
    raise ValueError(f"Unknown tool: {tool_name}")

In [0]:
import json

response = openai_client.chat.completions.create(
        model=LLM_ENDPOINT_NAME,
        messages=[{"role": "user", "content": "What is the square root of 429?"}],
        tools=builtin_tools,
    )

# Convert the ChatCompletion object to a dictionary
response_dict = response.model_dump()

# Pretty print as JSON
print(f"Response is {json.dumps(response_dict, indent=2)}")

msg = response.choices[0].message
if msg.tool_calls:
    call = msg.tool_calls[0]
    tool_result = call_tool(call.function.name, json.loads(call.function.arguments))
    print(f"Tool result: {tool_result.value}")
    print(f"Tool call ID: {call.id}")

In [0]:
def run_agent(prompt):
    """
    Send a user prompt to the LLM, and return a list of LLM response messages
    The LLM is allowed to call the code interpreter tool if needed, to respond to the user
    """
    result_msgs = []
    response = openai_client.chat.completions.create(
        model=LLM_ENDPOINT_NAME,
        messages=[{"role": "user", "content": prompt}],
        tools=builtin_tools,
    )
    msg = response.choices[0].message
    result_msgs.append(msg.to_dict())

    # If the model executed a tool, call it
    if msg.tool_calls:
        call = msg.tool_calls[0]
        tool_result = call_tool(call.function.name, json.loads(call.function.arguments))
        result_msgs.append(
            {
                "role": "tool",
                "content": tool_result.value,
                "name": call.function.name,
                "tool_call_id": call.id,
            }
        )
    return result_msgs

In [0]:
answer = run_agent("What is the square root of 429?")
for message in answer:
    print(f'{message["role"]}: {message["content"]}')

In [0]:
answer

## Prepare agent code for logging

Wrap your agent definition in MLflow’s [ChatAgent interface](https://mlflow.org/docs/latest/api_reference/python_api/mlflow.pyfunc.html#mlflow.pyfunc.ChatAgent) to prepare your code for logging.

By using MLflow’s standard agent authoring interface, you get built-in UIs for chatting with your agent and sharing it with others after deployment. ([AWS](https://docs.databricks.com/aws/en/generative-ai/agent-framework/author-agent#-use-chatagent-to-author-agents) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/agent-framework/author-agent) | [GCP](https://docs.databricks.com/gcp/en/generative-ai/agent-framework/author-agent))

In [0]:
import uuid
import mlflow
from typing import Any, Optional

from mlflow.pyfunc import ChatAgent
from mlflow.types.agent import ChatAgentMessage, ChatAgentResponse, ChatContext

mlflow.openai.autolog()

class QuickstartAgent(ChatAgent):
    def predict(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> ChatAgentResponse:
        # 1. Extract the last user prompt from the input messages
        prompt = messages[-1].content

        # 2. Call run_agent to get back a list of response messages
        raw_msgs = run_agent(prompt)

        # 3. Map each response message into a ChatAgentMessage and return
        # the response
        out = []
        for m in raw_msgs:
            out.append(ChatAgentMessage(id=uuid.uuid4().hex, **m))

        return ChatAgentResponse(messages=out)

In [0]:
AGENT = QuickstartAgent()
for response_message in AGENT.predict(
    {"messages": [{"role": "user", "content": "What's the square root of 429?"}]}
).messages:
    print(f"role: {response_message.role}, content: {response_message.content}")

## Log the agent

Log the agent and register it to Unity Catalog as a model ([AWS](https://docs.databricks.com/aws/en/machine-learning/manage-model-lifecycle/) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/machine-learning/manage-model-lifecycle/) | [GCP](https://docs.databricks.com/gcp/en/machine-learning/manage-model-lifecycle/)). This step packages the agent code and its dependencies into a single artifact to deploy it to a serving endpoint.

The following code cells do the following:

1. Copy the agent code from above and combine it into a single cell.
1. Add the `%%writefile` cell magic command at the top of the cell to save the agent code to a file called `quickstart_agent.py`.
1. Add a [mlflow.models.set_model()](https://mlflow.org/docs/latest/model#models-from-code) call to the bottom of the cell. This tells MLflow which Python agent object to use for making predictions when your agent is deployed.
1. Log the agent code in the `quickstart_agent.py` file using MLflow APIs ([AWS](https://docs.databricks.com/aws/en/generative-ai/agent-framework/log-agent) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/agent-framework/log-agent) | [GCP](https://docs.databricks.com/gcp/en/generative-ai/agent-framework/log-agent)).

In [0]:
%%writefile quickstart_agent.py

import json
import uuid
from databricks.sdk import WorkspaceClient
from databricks_openai import UCFunctionToolkit, DatabricksFunctionClient
from typing import Any, Optional

import mlflow
from mlflow.pyfunc import ChatAgent
from mlflow.types.agent import ChatAgentMessage, ChatAgentResponse, ChatContext

# Get an OpenAI client configured to talk to Databricks model serving endpoints
# We'll use this to query an LLM in our agent
openai_client = WorkspaceClient().serving_endpoints.get_open_ai_client()

# The snippet below tries to pick the first LLM API available in your Databricks workspace
# from a set of candidates. You can override and simplify it
# to just specify LLM_ENDPOINT_NAME.
LLM_ENDPOINT_NAME = None

def is_endpoint_available(endpoint_name):
  try:
    client = WorkspaceClient().serving_endpoints.get_open_ai_client()
    client.chat.completions.create(model=endpoint_name, messages=[{"role": "user", "content": "What is AI?"}])
    return True
  except Exception:
    return False
  
for candidate_endpoint_name in ["databricks-claude-3-7-sonnet", "databricks-meta-llama-3-3-70b-instruct"]:
    if is_endpoint_available(candidate_endpoint_name):
      LLM_ENDPOINT_NAME = candidate_endpoint_name
assert LLM_ENDPOINT_NAME is not None, "Please specify LLM_ENDPOINT_NAME"

# Enable automatic tracing of LLM calls
mlflow.openai.autolog()

# Load Databricks built-in tools (a stateless Python code interpreter tool)
client = DatabricksFunctionClient()
builtin_tools = UCFunctionToolkit(function_names=["system.ai.python_exec"], client=client).tools
for tool in builtin_tools:
    del tool["function"]["strict"]

def call_tool(tool_name, parameters):
    if tool_name == "system__ai__python_exec":
        return DatabricksFunctionClient().execute_function("system.ai.python_exec", parameters=parameters)
    raise ValueError(f"Unknown tool: {tool_name}")

def run_agent(prompt):
    """
    Send a user prompt to the LLM, and return a list of LLM response messages
    The LLM is allowed to call the code interpreter tool if needed, to respond to the user
    """
    result_msgs = []
    response = openai_client.chat.completions.create(
        model=LLM_ENDPOINT_NAME,
        messages=[{"role": "user", "content": prompt}],
        tools=builtin_tools,
    )
    msg = response.choices[0].message
    result_msgs.append(msg.to_dict())

    # If the model executed a tool, call it
    if msg.tool_calls:
        call = msg.tool_calls[0]
        tool_result = call_tool(call.function.name, json.loads(call.function.arguments))
        result_msgs.append({"role": "tool", "content": tool_result.value, "name": call.function.name, "tool_call_id": call.id})
    return result_msgs

class QuickstartAgent(ChatAgent):
    def predict(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> ChatAgentResponse:
        prompt = messages[-1].content
        raw_msgs = run_agent(prompt)
        out = []
        for m in raw_msgs:
            out.append(ChatAgentMessage(
                id=uuid.uuid4().hex,
                **m
            ))

        return ChatAgentResponse(messages=out)

AGENT = QuickstartAgent()
mlflow.models.set_model(AGENT)

In [0]:
dbutils.library.restartPython()

In [0]:
import mlflow
from mlflow.models.resources import DatabricksFunction, DatabricksServingEndpoint
from pkg_resources import get_distribution
from quickstart_agent import LLM_ENDPOINT_NAME

# Register the model to the workspace default catalog.
# Specify a catalog (e.g. "main") and schema name (e.g. "custom_schema") if needed,
# in order to register the agent to a different location
catalog_name = "agentic_ai"
schema_name = "databricks"
registered_model_name = f"{catalog_name}.{schema_name}.quickstart_agent"

# Specify Databricks product resources that the agent needs access to (our builtin python
# code interpreter tool and LLM serving endpoint), so that Databricks can automatically
# configure authentication for the agent to access these resources when it's deployed.
resources = [
    DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME),
    DatabricksFunction(function_name="system.ai.python_exec"),
]

mlflow.set_registry_uri("databricks-uc")
with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        artifact_path="agent",
        python_model="quickstart_agent.py",
        extra_pip_requirements=[
            f"databricks-connect=={get_distribution('databricks-connect').version}"
        ],
        resources=resources,
        registered_model_name=registered_model_name,
    )

## Deploy the agent

Run the cell below to deploy the agent ([AWS](https://docs.databricks.com/aws/en/generative-ai/agent-framework/deploy-agent) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/agent-framework/deploy-agent) | [GCP](https://docs.databricks.com/gcp/en/generative-ai/agent-framework/deploy-agent)). Once the agent endpoint starts, you can chat with it via AI Playground ([AWS](https://docs.databricks.com/aws/en/large-language-models/ai-playground) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/large-language-models/ai-playground) | [GCP](https://docs.databricks.com/gcp/en/large-language-models/ai-playground)), or share it with stakeholders ([AWS](https://docs.databricks.com/aws/en/generative-ai/agent-evaluation/review-app) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/agent-evaluation/review-app) | [GCP](https://docs.databricks.com/gcp/en/generative-ai/agent-evaluation/review-app)) for initial feedback, before sharing it more broadly.

In [0]:
from databricks import agents

deployment_info = agents.deploy(
    model_name=registered_model_name,
    model_version=logged_agent_info.registered_model_version,
    deploy_feedback_model=False,
)